In [10]:
import tensorflow as tf
import numpy as npy
import os
import sys

sys.path.insert(0, '../src')
import deepleeo.dataset.data_augment as dtaug

In [11]:
DATA_DIR = os.path.join(os.path.abspath(os.path.dirname("__file__")), '../', 'data_real', 'generated')
DATASET_FILE = os.path.join(DATA_DIR, 'samples_dataset.npz')

In [12]:
dataset = npy.load(DATASET_FILE)

print("Data Loaded:")
print("  -> Images: ", len(dataset["images"]))
print("  -> Labels: ", len(dataset["images"]))
print("  -> Classes: ", len(dataset["classes"]))

Data Loaded:
  -> Images:  5000
  -> Labels:  5000
  -> Classes:  4


In [13]:
angles = [90, 180, 270]
rotated_imgs = dtaug.rotate_images(dataset["images"], angles)
flipped_imgs = dtaug.flip_images(dataset["images"])

dataset["images"].extend(rotated_imgs)
dataset["images"].extend(flipped_imgs)

rotated_lbls = dtaug.rotate_images(dataset["labels"], angles)
flipped_lbls = dtaug.flip_images(dataset["labels"])

dataset["labels"].extend(rotated_lbls)
dataset["labels"].extend(flipped_lbls)

print("Data Augmentation Applied:")
print("  -> Images: ", len(dataset["images"]))
print("  -> Labels: ", len(dataset["images"]))

#TODO: Probably, to fix this error, will be necessary to implement normalization.
#TODO: Verify this error bellow:

TypeError: Images should have rank between 2 and 4.